In [17]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from datasets import load_dataset
import fastcore.all as fc
import torchvision.transforms as TF
import torch.nn.functional as F
from torcheval.metrics import MulticlassAccuracy, Mean
import matplotlib as mpl
from miniai.lec_9_v4 import *

mpl.rcParams['image.cmap'] = 'gray'

In [18]:
name = 'fashion_mnist'
x,y = 'image', 'label'
dd = load_dataset(name)

Found cached dataset fashion_mnist (/Users/arun/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
from torch.utils.data import DataLoader, default_collate
from operator import itemgetter

In [29]:
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

@inplace
def transform_(b):
    b[x] = [torch.flatten(TF.to_tensor(ele)) for ele in b[x]]

def get_dls(train_ds, valid_ds, batch_size, **kwargs):
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True, **kwargs),
        DataLoader(valid_ds, batch_size=batch_size*2, shuffle=False, **kwargs),
    )

def collate_dict(ds):
    g = itemgetter(*ds.features)
    def _f(b):
        return g(default_collate(b))
    return _f

class DataLoaders:
    def __init__(self, *dss):
        self.train,self.valid = dss[:2]
    
    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        cf = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), batch_size=batch_size, collate_fn=cf, **kwargs))

In [30]:
tds = dd.with_transform(transform_)

In [31]:
bs = 1024
dls = DataLoaders.from_dd(tds, batch_size=bs)

In [35]:
def conv(ni, nf, ks=3, act=True):
    res = nn.Conv2d(ni, nf, kernel_size=ks, stride=2, padding=ks//2)
    if res:
        res = nn.Sequential(res, nn.ReLU())
    return res

def cnn_layers():
    return [
        conv(1, 8, ks=5),
        conv(8, 16),
        conv(16, 32),
        conv(32, 64),
        conv(64, 10, act=False),
        nn.Flatten()
    ]